In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

In [2]:
cols=['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','target']
df=pd.read_csv('cleveland.data', names=cols)
# df=pd.read_csv('heart.csv')
#df['target'].nunique()
#df.head()
df.shape

(303, 14)

In [3]:
df['target'] = df['target'].replace([2,3,4],1)
#sns.countplot(x='target',data=df)
#df.dtypes
#df.head()
df.shape
#df['target'].nunique()

(303, 14)

In [4]:
df = df.replace('?', np.nan)
df.dropna(inplace=True)
df.shape

(297, 14)

### Attribute Information: ###

1. age                                                          2. sex      
3. chest pain type (4 values)                                   4. resting blood pressure         
5. serum cholestoral in mg/dl                                   6. fasting blood sugar > 120 mg/dl   
7. resting electrocardiographic results (values 0,1,2)          8. maximum heart rate achieved
9. exercise induced angina                                      10. oldpeak = ST depression induced by exercise relative to rest  11. the slope of the peak exercise ST segment                   12. number of major vessels (0-3) colored by flourosopy       13. thal: 0 = normal; 1 = fixed defect; 2 = reversable defect   14. Target

In [5]:
df['ca'] = df['ca'].astype(float)
df['thal'] = df['thal'].astype(float)
output=df.target
data=df.drop('target',axis=1)
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0


In [117]:
# #Lasso Feature Selection

# from sklearn import linear_model
# regressor = linear_model.Lasso(alpha=0.15)
# regressor.fit(data, output)
# weights1=regressor.coef_
# weights1=[abs(i) for i in weights1]
# cols=['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal']
# lasso = pd.DataFrame()
# lasso['features']=cols
# lasso['lasso']=weights1
# #lasso=lasso.sort_values('lasso',ascending=False)
# lasso.head()
# regressor.coef_




In [6]:
#Information Gain Feature selection

from sklearn.feature_selection import mutual_info_classif
import matplotlib.pyplot as plt
%matplotlib inline

weights1=mutual_info_classif(data,output)


cols=['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal']
mi = pd.DataFrame()
mi['features']=cols
mi['Mutaul_Info']=weights1
#lasso=lasso.sort_values('lasso',ascending=False)
mi.head()

,features,Mutaul_Info
0,age,0.003912
1,sex,0.065385
2,cp,0.174688
3,trestbps,0.000000
4,chol,0.060142


In [7]:
#MRMR Feature selection

data1=data.assign(target=output)
relation=data1.corr()
cols=['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal']
weight=[]
mrmr= pd.DataFrame()
for feature in cols:
    rel=abs(relation[feature]['target'])
    red=0
    for feature2 in cols:
        if(feature!=feature2):
            red=red+abs(relation[feature][feature2])
    weight.append(rel/red)
mrmr['features']=cols
mrmr['mrmr']=weight
#mrmr=mrmr.sort_values('mrmr',ascending=False)
mrmr.head()

,features,mrmr
0,age,0.098116
1,sex,0.221348
2,cp,0.212404
3,trestbps,0.100995
4,chol,0.079010


In [8]:
import sklearn_relief as relief

X = np.array(data)
Y = np.array(output)

r = relief.Relief(n_features=10)

my_transformed_matrix = r.fit_transform(X,Y)
weights3=r.w_
relief= pd.DataFrame()
relief['features']=cols
relief['relief']=weights3
relief.head()

,features,relief
0,age,5.18
1,sex,0.37
2,cp,1.01
3,trestbps,6.50
4,chol,6.60


In [9]:
norm_wt1  = [float(i)/sum(weights1) for i in weights1]
norm_wt2  = [float(i)/sum(weight) for i in weight]
norm_wt3  = [float(i)/sum(weights3) for i in weights3]
Ensembled_weights=[]
for i in range(len(norm_wt1)):
    k=(norm_wt2[i]+norm_wt2[i]+norm_wt3[i])/3
    Ensembled_weights.append(k);
#Ensembled_weights
Final_weights= pd.DataFrame()
Final_weights['features']=cols
Final_weights['ensemble']=Ensembled_weights
Final_weights=Final_weights.sort_values('ensemble',ascending=False)
Final_Features=list(Final_weights.head(10).features)
Final_Features

['thalach',
 'trestbps',
 'chol',
 'thal',
 'age',
 'cp',
 'ca',
 'sex',
 'exang',
 'oldpeak']

In [10]:
final_df=data[Final_Features]
final_df.head()
# data=final_df
# data

,thalach,trestbps,chol,thal,age,cp,ca,sex,exang,oldpeak
0,150.0,145.0,233.0,6.0,63.0,1.0,0.0,1.0,0.0,2.3
1,108.0,160.0,286.0,3.0,67.0,4.0,3.0,1.0,1.0,1.5
2,129.0,120.0,229.0,7.0,67.0,4.0,2.0,1.0,1.0,2.6
3,187.0,130.0,250.0,3.0,37.0,3.0,0.0,1.0,0.0,3.5
4,172.0,130.0,204.0,3.0,41.0,2.0,0.0,0.0,0.0,1.4


In [123]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# data = scaler.fit_transform(data)
# data = pd.DataFrame(data,columns=cols)
# data
data1=final_df
#min_max normalization
data=(final_df-final_df.min())/(final_df.max()-final_df.min())
data.head()

,thalach,trestbps,chol,thal,age,cp,sex,ca,exang,oldpeak
0,0.603053,0.481132,0.244292,0.75,0.708333,0.000000,1.0,0.000000,0.0,0.370968
1,0.282443,0.622642,0.365297,0.00,0.791667,1.000000,1.0,1.000000,1.0,0.241935
2,0.442748,0.245283,0.235160,1.00,0.791667,1.000000,1.0,0.666667,1.0,0.419355
3,0.885496,0.339623,0.283105,0.00,0.166667,0.666667,1.0,0.000000,0.0,0.564516
4,0.770992,0.339623,0.178082,0.00,0.250000,0.333333,0.0,0.000000,0.0,0.225806


In [124]:
x_train,x_test,y_train,y_test=train_test_split(data,output,test_size=0.2,random_state=42)

In [ ]:
from sklearn.model_selection import GridSearchCV
LR = LogisticRegression()
LRparam_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'penalty': ['l2'],
    'max_iter': list(range(100,800,100)),
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}
LR_search = GridSearchCV(LR, param_grid=LRparam_grid, refit = True, verbose = 3, cv=5)

# fitting the model for grid search 
LR_search.fit(x_train , y_train)
LR_search.best_params_
# summarize
print('Mean Accuracy: %.3f' % LR_search.best_score_)
print('Config: %s' % LR_search.best_params_)

Fitting 5 folds for each of 245 candidates, totalling 1225 fits
[CV 1/5] END C=0.001, max_iter=100, penalty=l2, solver=newton-cg;, score=0.521 total time=   0.0s
[CV 2/5] END C=0.001, max_iter=100, penalty=l2, solver=newton-cg;, score=0.521 total time=   0.0s
[CV 3/5] END C=0.001, max_iter=100, penalty=l2, solver=newton-cg;, score=0.532 total time=   0.0s
[CV 4/5] END C=0.001, max_iter=100, penalty=l2, solver=newton-cg;, score=0.532 total time=   0.0s
[CV 5/5] END C=0.001, max_iter=100, penalty=l2, solver=newton-cg;, score=0.511 total time=   0.0s
[CV 1/5] END C=0.001, max_iter=100, penalty=l2, solver=lbfgs;, score=0.521 total time=   0.0s
[CV 2/5] END C=0.001, max_iter=100, penalty=l2, solver=lbfgs;, score=0.521 total time=   0.0s
[CV 3/5] END C=0.001, max_iter=100, penalty=l2, solver=lbfgs;, score=0.532 total time=   0.0s
[CV 4/5] END C=0.001, max_iter=100, penalty=l2, solver=lbfgs;, score=0.532 total time=   0.0s
[CV 5/5] END C=0.001, max_iter=100, penalty=l2, solver=lbfgs;, score=0

[CV 3/5] END C=0.001, max_iter=500, penalty=l2, solver=liblinear;, score=0.617 total time=   0.0s
[CV 4/5] END C=0.001, max_iter=500, penalty=l2, solver=liblinear;, score=0.553 total time=   0.0s
[CV 5/5] END C=0.001, max_iter=500, penalty=l2, solver=liblinear;, score=0.766 total time=   0.0s
[CV 1/5] END C=0.001, max_iter=500, penalty=l2, solver=sag;, score=0.521 total time=   0.0s
[CV 2/5] END C=0.001, max_iter=500, penalty=l2, solver=sag;, score=0.521 total time=   0.0s
[CV 3/5] END C=0.001, max_iter=500, penalty=l2, solver=sag;, score=0.532 total time=   0.0s
[CV 4/5] END C=0.001, max_iter=500, penalty=l2, solver=sag;, score=0.532 total time=   0.0s
[CV 5/5] END C=0.001, max_iter=500, penalty=l2, solver=sag;, score=0.511 total time=   0.0s
[CV 1/5] END C=0.001, max_iter=500, penalty=l2, solver=saga;, score=0.521 total time=   0.0s
[CV 2/5] END C=0.001, max_iter=500, penalty=l2, solver=saga;, score=0.521 total time=   0.0s
[CV 3/5] END C=0.001, max_iter=500, penalty=l2, solver=saga;

[CV 1/5] END C=0.01, max_iter=300, penalty=l2, solver=lbfgs;, score=0.812 total time=   0.0s
[CV 2/5] END C=0.01, max_iter=300, penalty=l2, solver=lbfgs;, score=0.750 total time=   0.0s
[CV 3/5] END C=0.01, max_iter=300, penalty=l2, solver=lbfgs;, score=0.830 total time=   0.0s
[CV 4/5] END C=0.01, max_iter=300, penalty=l2, solver=lbfgs;, score=0.787 total time=   0.0s
[CV 5/5] END C=0.01, max_iter=300, penalty=l2, solver=lbfgs;, score=0.787 total time=   0.0s
[CV 1/5] END C=0.01, max_iter=300, penalty=l2, solver=liblinear;, score=0.729 total time=   0.0s
[CV 2/5] END C=0.01, max_iter=300, penalty=l2, solver=liblinear;, score=0.771 total time=   0.0s
[CV 3/5] END C=0.01, max_iter=300, penalty=l2, solver=liblinear;, score=0.851 total time=   0.0s
[CV 4/5] END C=0.01, max_iter=300, penalty=l2, solver=liblinear;, score=0.681 total time=   0.0s
[CV 5/5] END C=0.01, max_iter=300, penalty=l2, solver=liblinear;, score=0.745 total time=   0.0s
[CV 1/5] END C=0.01, max_iter=300, penalty=l2, sol

[CV 1/5] END C=0.01, max_iter=700, penalty=l2, solver=newton-cg;, score=0.812 total time=   0.0s
[CV 2/5] END C=0.01, max_iter=700, penalty=l2, solver=newton-cg;, score=0.750 total time=   0.0s
[CV 3/5] END C=0.01, max_iter=700, penalty=l2, solver=newton-cg;, score=0.830 total time=   0.0s
[CV 4/5] END C=0.01, max_iter=700, penalty=l2, solver=newton-cg;, score=0.787 total time=   0.0s
[CV 5/5] END C=0.01, max_iter=700, penalty=l2, solver=newton-cg;, score=0.787 total time=   0.0s
[CV 1/5] END C=0.01, max_iter=700, penalty=l2, solver=lbfgs;, score=0.812 total time=   0.0s
[CV 2/5] END C=0.01, max_iter=700, penalty=l2, solver=lbfgs;, score=0.750 total time=   0.0s
[CV 3/5] END C=0.01, max_iter=700, penalty=l2, solver=lbfgs;, score=0.830 total time=   0.0s
[CV 4/5] END C=0.01, max_iter=700, penalty=l2, solver=lbfgs;, score=0.787 total time=   0.0s
[CV 5/5] END C=0.01, max_iter=700, penalty=l2, solver=lbfgs;, score=0.787 total time=   0.0s
[CV 1/5] END C=0.01, max_iter=700, penalty=l2, sol

[CV 1/5] END C=0.1, max_iter=500, penalty=l2, solver=newton-cg;, score=0.792 total time=   0.0s
[CV 2/5] END C=0.1, max_iter=500, penalty=l2, solver=newton-cg;, score=0.792 total time=   0.0s
[CV 3/5] END C=0.1, max_iter=500, penalty=l2, solver=newton-cg;, score=0.872 total time=   0.0s
[CV 4/5] END C=0.1, max_iter=500, penalty=l2, solver=newton-cg;, score=0.702 total time=   0.0s
[CV 5/5] END C=0.1, max_iter=500, penalty=l2, solver=newton-cg;, score=0.851 total time=   0.0s
[CV 1/5] END C=0.1, max_iter=500, penalty=l2, solver=lbfgs;, score=0.792 total time=   0.0s
[CV 2/5] END C=0.1, max_iter=500, penalty=l2, solver=lbfgs;, score=0.792 total time=   0.0s
[CV 3/5] END C=0.1, max_iter=500, penalty=l2, solver=lbfgs;, score=0.872 total time=   0.0s
[CV 4/5] END C=0.1, max_iter=500, penalty=l2, solver=lbfgs;, score=0.702 total time=   0.0s
[CV 5/5] END C=0.1, max_iter=500, penalty=l2, solver=lbfgs;, score=0.851 total time=   0.0s
[CV 1/5] END C=0.1, max_iter=500, penalty=l2, solver=libline

...

C:\Users\anilc\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 5/5] END C=100, max_iter=300, penalty=l2, solver=lbfgs;, score=0.851 total time=   0.0s
[CV 1/5] END C=100, max_iter=300, penalty=l2, solver=liblinear;, score=0.812 total time=   0.0s
[CV 2/5] END C=100, max_iter=300, penalty=l2, solver=liblinear;, score=0.812 total time=   0.0s
[CV 3/5] END C=100, max_iter=300, penalty=l2, solver=liblinear;, score=0.872 total time=   0.0s
[CV 4/5] END C=100, max_iter=300, penalty=l2, solver=liblinear;, score=0.723 total time=   0.0s
[CV 5/5] END C=100, max_iter=300, penalty=l2, solver=liblinear;, score=0.851 total time=   0.0s
[CV 1/5] END C=100, max_iter=300, penalty=l2, solver=sag;, score=0.812 total time=   0.0s
[CV 2/5] END C=100, max_iter=300, penalty=l2, solver=sag;, score=0.812 total time=   0.0s
[CV 3/5] END C=100, max_iter=300, penalty=l2, solver=sag;, score=0.872 total time=   0.0s
[CV 4/5] END C=100, max_iter=300, penalty=l2, solver=sag;, score=0.723 total time=   0.0s
[CV 5/5] END C=100, max_iter=300, penalty=l2, solver=sag;, score=0.8

[CV 3/5] END C=100, max_iter=700, penalty=l2, solver=newton-cg;, score=0.872 total time=   0.0s
[CV 4/5] END C=100, max_iter=700, penalty=l2, solver=newton-cg;, score=0.723 total time=   0.0s
[CV 5/5] END C=100, max_iter=700, penalty=l2, solver=newton-cg;, score=0.851 total time=   0.0s
[CV 1/5] END C=100, max_iter=700, penalty=l2, solver=lbfgs;, score=0.812 total time=   0.0s
[CV 2/5] END C=100, max_iter=700, penalty=l2, solver=lbfgs;, score=0.812 total time=   0.0s
[CV 3/5] END C=100, max_iter=700, penalty=l2, solver=lbfgs;, score=0.872 total time=   0.0s
[CV 4/5] END C=100, max_iter=700, penalty=l2, solver=lbfgs;, score=0.723 total time=   0.0s
[CV 5/5] END C=100, max_iter=700, penalty=l2, solver=lbfgs;, score=0.851 total time=   0.0s
[CV 1/5] END C=100, max_iter=700, penalty=l2, solver=liblinear;, score=0.812 total time=   0.0s
[CV 2/5] END C=100, max_iter=700, penalty=l2, solver=liblinear;, score=0.812 total time=   0.0s
[CV 3/5] END C=100, max_iter=700, penalty=l2, solver=libline

C:\Users\anilc\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\anilc\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 2/5] END C=1000, max_iter=100, penalty=l2, solver=saga;, score=0.833 total time=   0.0s
[CV 3/5] END C=1000, max_iter=100, penalty=l2, solver=saga;, score=0.872 total time=   0.0s
[CV 4/5] END C=1000, max_iter=100, penalty=l2, solver=saga;, score=0.723 total time=   0.0s
[CV 5/5] END C=1000, max_iter=100, penalty=l2, solver=saga;, score=0.851 total time=   0.0s
[CV 1/5] END C=1000, max_iter=200, penalty=l2, solver=newton-cg;, score=0.812 total time=   0.0s
[CV 2/5] END C=1000, max_iter=200, penalty=l2, solver=newton-cg;, score=0.833 total time=   0.0s
[CV 3/5] END C=1000, max_iter=200, penalty=l2, solver=newton-cg;, score=0.872 total time=   0.0s
[CV 4/5] END C=1000, max_iter=200, penalty=l2, solver=newton-cg;, score=0.723 total time=   0.0s
[CV 5/5] END C=1000, max_iter=200, penalty=l2, solver=newton-cg;, score=0.851 total time=   0.0s
[CV 1/5] END C=1000, max_iter=200, penalty=l2, solver=lbfgs;, score=0.812 total time=   0.0s
[CV 2/5] END C=1000, max_iter=200, penalty=l2, solver=

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import BaggingClassifier

pipeline1 = LogisticRegression(random_state = 1,C= 1000, penalty= 'l2',max_iter =100, solver = 'newton-cg')
#pipeline = make_pipeline(StandardScaler(),
#                        LogisticRegression(random_state=1))
bgclassifier1 = BaggingClassifier(base_estimator=pipeline1, n_estimators=100,
                                 max_features=8,
                                 max_samples=100,
                                 random_state=1, n_jobs=5)
bgclassifier1.fit(x_train, y_train)

# print('Model test Score: %.3f, ' %bgclassifier1.score(x_test, y_test),
#       'Model training Score: %.3f' %bgclassifier1.score(x_train, y_train))
y_pred = bgclassifier1.predict(x_test)
ac = accuracy_score(y_test, y_pred)
ac

In [ ]:
#GridsearchCV for decision tree
from sklearn.tree import DecisionTreeClassifier

param_grid = {
              'ccp_alpha': [0.1, .01, .001],
              'max_depth' : [5, 6, 7, 8, 9],
              'criterion' :['gini', 'entropy']
             }
tree_clas = DecisionTreeClassifier(random_state=1024)
dt_search = GridSearchCV(estimator=tree_clas, param_grid=param_grid, cv=5, verbose=True)
dt_search.fit(x_train, y_train)

print('Mean Accuracy: %.3f' % dt_search.best_score_)
print('Config: %s' % dt_search.best_params_)

In [ ]:
pipeline2 = DecisionTreeClassifier(random_state = 10,ccp_alpha= 0.01, criterion= 'entropy', max_depth= 5)
#pipeline = make_pipeline(StandardScaler(),
#                        LogisticRegression(random_state=1))
bgclassifier2 = BaggingClassifier(base_estimator=pipeline2, n_estimators=100,
                                 max_features=8,
                                 max_samples=100,
                                 random_state=1, n_jobs=5)
bgclassifier2.fit(x_train, y_train)

# print('Model test Score: %.3f, ' %bgclassifier2.score(x_test, y_test),
#       'Model training Score: %.3f' %bgclassifier2.score(x_train, y_train))

y_pred1 = bgclassifier2.predict(x_test)
ac1 = accuracy_score(y_test, y_pred1)
ac1

In [ ]:
k_range = list(range(3, 31))
param_grid = dict(n_neighbors=k_range)
knn_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, verbose=True)
knn_search.fit(x_train, y_train)
print('Mean Accuracy: %.3f' % knn_search.best_score_)
print('Config: %s' % knn_search.best_params_)

In [ ]:
pipeline3 = KNeighborsClassifier(n_neighbors=71)
#pipeline = make_pipeline(StandardScaler(),
#                        LogisticRegression(random_state=1))
bgclassifier3 = BaggingClassifier(base_estimator=pipeline3, n_estimators=100,
                                 max_features=8,
                                 max_samples=100,
                                 random_state=1, n_jobs=5)
bgclassifier3.fit(x_train, y_train)

# print('Model test Score: %.3f, ' %bgclassifier3.score(x_test, y_test),
#       'Model training Score: %.3f' %bgclassifier3.score(x_train, y_train))

y_pred2 = bgclassifier3.predict(x_test)
ac2 = accuracy_score(y_test, y_pred2)
ac2

In [ ]:
# #GridsearchCV for decision tree
# from sklearn.tree import DecisionTreeClassifier

# param_grid = {
#               'ccp_alpha': [0.1, .01, .001],
#               'max_depth' : [5, 6, 7, 8, 9],
#               'criterion' :['gini', 'entropy']
#              }
# tree_clas = DecisionTreeClassifier(random_state=1024)
# dt_search = GridSearchCV(estimator=tree_clas, param_grid=param_grid, cv=5, verbose=True)
# dt_search.fit(x_train, y_train)

# print('Mean Accuracy: %.3f' % dt_search.best_score_)
# print('Config: %s' % dt_search.best_params_)

In [ ]:
from sklearn.ensemble import StackingClassifier
clf1 = bgclassifier1
clf2 = bgclassifier2
clf3 = bgclassifier3
lr = LogisticRegression(random_state = 1,C= 1000, penalty= 'l2',max_iter =100, solver = 'newton-cg')
estimators = [('lr', clf1),('dt', clf2),('knn',clf3),('lrr',pipeline1),('dtt',pipeline2),('knnn',pipeline3)]
clf = StackingClassifier(estimators=estimators, final_estimator=lr)
clf.fit(x_train,y_train)
y_pred3 = clf.predict(x_test)
cm3 = confusion_matrix(y_test, y_pred3)
ac3 = accuracy_score(y_test, y_pred3)
ac3

In [ ]:
acc=[ac,ac1,ac2,ac3]
print(acc)